In [1]:
import json
from collections import defaultdict
import csv

path = "adapter_eval_result.raw"

best_per_rc = {}  # key=(round, client_id) -> {'val_loss','test_acc','adapter_idx'}

with open(path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        rec = json.loads(line)
        rr = rec.get("Results_raw", {})

        rnd = rec.get("Round", rr.get("round"))
        cid = rr.get("client_id")
        val_loss = rr.get("val_loss")
        test_acc = rr.get("test_acc")
        adapter_idx = rr.get("adapter_idx")

        if rnd is None or cid is None or val_loss is None or test_acc is None:
            continue

        key = (rnd, cid)
        cur = best_per_rc.get(key)
        # tie-breaker: val_loss 동일하면 test_acc가 큰 것 선택
        if (cur is None
            or (val_loss < cur["val_loss"] - 1e-12)
            or (abs(val_loss - cur["val_loss"]) <= 1e-12 and test_acc > cur["test_acc"])):
            best_per_rc[key] = {
                "val_loss": val_loss,
                "test_acc": test_acc,
                "adapter_idx": adapter_idx
            }

round_to_accs = defaultdict(list)
for (rnd, _cid), info in best_per_rc.items():
    round_to_accs[rnd].append(info["test_acc"])

rows = []
for rnd in sorted(round_to_accs):
    accs = round_to_accs[rnd]
    mean_acc = (sum(accs) / len(accs)) if accs else float("nan")  # ← fmean 대신
    rows.append({"round": rnd, "n_clients": len(accs), "mean_test_acc": mean_acc})

for r in rows:
    print(f"Round {r['round']:>4}: n={r['n_clients']:>3}  mean_test_acc={r['mean_test_acc']:.6f}")

with open("mean_test_acc_per_round.csv", "w", newline="", encoding="utf-8") as wf:
    writer = csv.DictWriter(wf, fieldnames=["round", "n_clients", "mean_test_acc"])
    writer.writeheader()
    writer.writerows(rows)
print("Saved: mean_test_acc_per_round.csv")


Round    0: n= 53  mean_test_acc=0.533019
Round   25: n= 53  mean_test_acc=0.540566
Round   50: n= 53  mean_test_acc=0.552358
Round   75: n= 53  mean_test_acc=0.554717
Round  100: n= 53  mean_test_acc=0.576415
Round  125: n= 53  mean_test_acc=0.578302
Round  150: n= 53  mean_test_acc=0.577358
Saved: mean_test_acc_per_round.csv
